# <span style = "color :  #ca6f1e ">Activité Python : Dynamique des populations - Modèle de Verhulst, Partie III </span>
## introduction au chaos et fractales de Liapounov

Bienvenue dans cette première partie du modèle de Verhulst ! 

Cet outil va te permettre d'exécuter du code Python facilement, à travers ce que j'ai déjà codé. 
 
Tu verras que beaucoup de commandes et lignes de codes sont très avancées. Mais ne t'inquiète pas, l'objectif n'est pas que tu comprennes tout le code mais plutôt que tu découvres la puissance de l'informatique pour ce genre de calculs et pour l'intuition qu'elle peut t'apporter, mais aussi ses limites. Tu auras donc à ta disposition un outil pour modifier les paramètres des simulations et essayer de comprendre l'influence des paramètres sur l'évolution. Bonne découverte !




<b>Remarque</b> : au début, appuie sur Cell->Run All pour tout lancer. Pour activer une cellule de code particulière (comme après une modification), clique dessus et appuie sur Cells->Run Cells, ou Shift+Enter ;)

In [8]:
import os
import sys
import random as rd
import math
import numpy as np
import skimage.io
import matplotlib
import matplotlib.pyplot as plt
import time

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column, row
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from bokeh.plotting import ColumnDataSource, output_file, show

output_notebook(hide_banner=True)

def f_mu(mu):
    return lambda x:mu*x*(1-x)

f_4 = f_mu(4)

### $\underline{\text{Un chaos compréhensible : détermination aléatoire d'un antécédent de tout }x\in[0,1]}$

Nous avons pu voir que la dynamique de $f_4$ est chaotique. Néanmoins, le chaos qui y est associé répond à un certain comportement "prévisible". Nous allons donc l'illustrer ici grâce à un algorithme. 

Celui-ci va nous permettre, pour tous $x\in[0,1]$, $\epsilon > 0$ et $N\in \mathbb{N}^{*}$ de trouver un $u_0$ tel que : 
$$
f^N_4(u_0)\in[x-\epsilon, x+\epsilon]
$$

Ainsi, malgré la sensibilité aux conditions initiales, nous pouvons trouver une valeur initiale dont nous connaissons, à une précision choisie, la valeur de la suite associée à une itération choisie.

In [9]:
#Un chaos prédictible
def convergence_vers_x(x,preci,nb_etape):
    print("On note :")
    print("- x la valeur que l'on souhaite atteindre")
    print("- preci l'entier tel que l'on souhaite atteindre x à 10^{-preci} près")
    print("- nb_etape le nombre d'étapes de la convergence")
    p=preci
    preci=int(p*np.log(10)/np.log(2))+2
    a=np.arcsin(np.sqrt(x))/np.pi
    b=int((2**preci)*a)
    c=rd.randint(0,2**nb_etape)/(2**nb_etape)+b/(2**(nb_etape+preci))
    liste_valeurs=[np.sin(c*np.pi)**2]
    for i in range(nb_etape):
        liste_valeurs.append(f_4(liste_valeurs[-1]))
    print("A partir de u0={}, la valeur finale obtenue est de {} au lieu de {} après {} étapes.".format(liste_valeurs[0], liste_valeurs[-1], x, nb_etape))
    
    fig = figure(width=470, height=300, title="Convergence vers la valeur choisie")
    fig.line(list(range(nb_etape+1)), liste_valeurs, legend_label="Evolution de la suite", color='blue')
    fig.x([nb_etape],[x],legend_label="Valeur attendue", color="red")
    fig.legend.location = "top_right" 
    
    fig1 = figure(width=470, height=300, y_axis_type="log", title="Ecart de la convergence vers la valeur choisie")
    fig1.line([0,nb_etape],[10**(-p), 10**(-p)], legend_label="Précision attendue", color="black")
    fig1.x(list(range(nb_etape+1)), abs(x-np.array(liste_valeurs)), legend_label="Evolution de l'écart à x", color='blue')
    fig1.legend.location = "bottom_left" 
    show(row(fig, fig1))

In [10]:
interact(convergence_vers_x, x=widgets.FloatSlider(min=0, max=1, step=0.05, value=0.5, continuous_update=False), preci=widgets.IntSlider(min=1, max=15, step=1, value=4, continuous_update=False), nb_etape=widgets.IntSlider(min=1, max=50, step=1, value=10, continuous_update=False))

interactive(children=(FloatSlider(value=0.5, continuous_update=False, description='x', max=1.0, step=0.05), In…

<function __main__.convergence_vers_x(x, preci, nb_etape)>

On peut observer que les résultats obtenus correspondent bien en général aux conditions décidées lorsque $nb_{etape}\leq 20$. En effet, pour $nb_{etape}\geq 20$, les erreurs de calcul successives, alliées à la sensibilité aux conditions initiales, mènent à l'obtention de résultats pas aussi précis que ceux qui étaient atendus. Néanmoins, dans un cadre théorique, cet algorithme fonctionne à tous les coups : c'est la limite entre la théorie et la réalité matérielle !

### $\underline{\text{Le diagramme de bifurcation complet}}$

Nous sommes enfin prêts pour admirer le diagramme de bifurcation complet : les points fixes attractifs, puis le cascade de doublement de périodes, et enfin toutes les choses étranges qui se passent après !

In [4]:
def derniers_elements(mu, x0, epsi):
    liste=[x0]
    x=x0
    for k in range(200):
        x=f_mu(mu)(x)
        liste.append(x)
    if mu<=3:
        return np.array(liste[-1:])
    for k in range(1,200):
        if abs(liste[-1-k]-liste[-1])<epsi:
            return np.array(liste[-k:])
        
def affichage_global(nb_points):
    print("Attention, le calcul peut être un peu long avec un grand nombre de points !")
    l=np.linspace(0,4,nb_points)
    liste_x=[]
    liste_y=[]
    plt.close()
    for mu in l:
        try:
            cycle=derniers_elements(mu,0.1,0.001)
            for k in range(cycle.shape[0]):
                liste_x.append(mu)
                liste_y.append(cycle[k])
        except:
            pass
    fig = figure(width=900, height=500, title="Diagramme de bifurcation")
    fig.x(liste_x,liste_y, size=1)
    show(fig)

In [5]:
interact(affichage_global, nb_points=widgets.IntSlider(min=2000, max=100000, step=1000, value=2000, continuous_update=False))

interactive(children=(IntSlider(value=2000, continuous_update=False, description='nb_points', max=100000, min=…

<function __main__.affichage_global(nb_points)>

N'hésite pas à utiliser la fonction Zoom pour regarder plus précisément ce qu'il se passe entre 3 et 4 !

### $\underline{\text{Les fractales de Liapounov}}$

Et pour finir en beauté, je te propose ici de voir par toi-même l'obtention de fractale de Liapounov, créée en utilisant la suite logistique ! Et si tu veux avoir plus de précisions sur les idées qui se cachent derrière leur obtention, n'hésite pas à aller jeter un oeil à la dernière partie du PDF !

In [6]:
def expo_lyapunov(a,b,lim):
    x=0.5
    l=0
    x=f_mu(a)(x)
    l+=np.log(abs(b*(1-2*x)))
    x=f_mu(b)(x)
    for i in range(1,lim):
        l+=np.log(abs(a*(1-2*x)))
        x=f_mu(a)(x)
        l+=np.log(abs(b*(1-2*x)))
        x=f_mu(b)(x)
    return l/(2*lim-1)

def fractale(nb):
    print("Attention, le temps de calcul peut être très important !\nEn cas de problème, appuie sur Kernel-> Restart & Run All pour tout relancer")
    lx=np.linspace(2,4,nb)
    ly=np.linspace(2,4,nb)
    tab=np.zeros((nb,nb),dtype=float)
    t=time.time()
    for i in range(nb):
        for j in range(nb):
            tab[i,j]=expo_lyapunov(lx[i],ly[j],40)
        if i%(nb//100)==0: print("■", end="")
        if i==2*nb//100:
            print("Temps restant estimé : ",int(49*(time.time()-t))," s")
    plt.close()
    fig = plt.figure(frameon=False)
    fig.set_size_inches(50,50)
    #fig.savefig("chemin_d_acces\\fractale.png",dpi=300)
    plt.imshow(tab,"inferno")
    fig.show()

In [7]:
interact(fractale, nb=widgets.IntSlider(min=100, max=5000, step=100, value=200, continuous_update=False))

interactive(children=(IntSlider(value=200, continuous_update=False, description='nb', max=5000, min=100, step=…

<function __main__.fractale(nb)>